In [56]:
from Models import *
from Utils import *
from FastModels import *
import warnings
from livelossplot import PlotLossesKeras
# warnings.filterwarnings('ignore')

In [57]:
from gensim.models import KeyedVectors
import sentencepiece as spm

max_len = 15
enablePadding = True

sp = spm.SentencePieceProcessor()
sp.Load('/work/data/bpe/en.wiki.bpe.op50000.model')
bpe = KeyedVectors.load_word2vec_format("/work/data/bpe/en.wiki.bpe.op50000.d200.w2v.bin", binary=True)
bpe.index2word = [''] + bpe.index2word + ['<sos>'] + ['<eos>']  # add empty string 
nb_words = len(bpe.index2word)
# word2index
bpe_dict = {bpe.index2word[i]: i for i in range(len(bpe.index2word))}
# construct embedding_matrix
embedding_matrix = np.concatenate([np.zeros((1, bpe.vector_size)), bpe.vectors, np.zeros((2, bpe.vector_size))]) # add zero vector for empty string (i.e. used for padding)

embedding_layer = Embedding(nb_words,
                    embedding_matrix.shape[-1],
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=True)

In [58]:
enablePadding = True
df_july, qrel_july = get_test_data("JulyFlower", "/work/")
q_july = parse_texts_bpe(df_july.q.tolist(), sp, bpe_dict, max_len, enablePadding, "post")
d_july = parse_texts_bpe(df_july.d.tolist(), sp, bpe_dict, max_len, enablePadding, "post")

In [59]:
class Seq2Seq(object):
    
    def __init__(self, nb_words, max_len, embedding_matrix, dim, optimizer=Adam(), keep_rate_word_dropout=0.5):
        self.dim = dim
        self.nb_words = nb_words
        self.max_len = max_len
        self.embedding_matrix = embedding_matrix
        self.optimizer = optimizer
        self.keep_rate_word_dropout = keep_rate_word_dropout
        self.build()

    def build(self):
        hidden_dim = self.dim[0]
        latent_dim = self.dim[1]

        # Define an input sequence and process it.
        encoder_inputs = Input(shape=(self.max_len,))
        kl_inputs = Input(shape=(1,))
        encoder_embedding = Embedding(self.nb_words,
                                        self.embedding_matrix.shape[-1],
                                        weights=[self.embedding_matrix],
                                        input_length=self.max_len,
                                        mask_zero=True,
                                        trainable=True)

        encoder_lstm = GRU(hidden_dim, return_state=True)

        x = encoder_embedding(encoder_inputs)
        _, state = encoder_lstm(x)

        mean = Dense(latent_dim)
        var = Dense(latent_dim)

        state_mean = mean(state)
        state_var = var(state)

        def sampling(args):
            z_mean, z_log_var = args
            epsilon = K.random_normal(shape=(K.shape(z_mean)[0], K.shape(z_mean)[1]), mean=0.,\
                                      stddev=1)
            return z_mean + K.exp(z_log_var / 2) * epsilon 

        state_z = Lambda(sampling, name="kl")([state_mean, state_var])
        
        decoder_inputs = Input(shape=(self.max_len,))
        
        
        latent2hidden = Dense(hidden_dim)
        decoder_lstm = GRU(hidden_dim, return_sequences=True)
        decoder_dense = Dense(self.nb_words, activation='softmax', name="rec")
        
        
        x = encoder_embedding(decoder_inputs)
        decoder_outputs = decoder_lstm(x, initial_state=latent2hidden(state_z))
        rec_outputs = decoder_dense(decoder_outputs)

        def kl_loss(x, x_):
            kl_loss = - 0.5 * K.sum(1 + state_var - K.square(state_mean) - K.exp(state_var), axis=-1)
            return kl_inputs * kl_loss

        self.model = Model([encoder_inputs, decoder_inputs, kl_inputs], [rec_outputs, state_z])
        self.model.compile(optimizer=self.optimizer, loss=['sparse_categorical_crossentropy', kl_loss])
        self.encoder = Model(encoder_inputs, state_mean)

    def name(self):
        return "seq2seq"
    
    
    def word_dropout(self, x, unk_token):

        x_ = np.copy(x)
        rows, cols = np.nonzero(x_)
        for r, c in zip(rows, cols):
            if random.random() <= self.keep_rate_word_dropout:
                continue
            x_[r][c] = unk_token

        return x_


In [60]:
run = Seq2Seq(nb_words, max_len, embedding_matrix, [256,16])

In [62]:
run.model.inputs

[<tf.Tensor 'input_9:0' shape=(?, 15) dtype=float32>,
 <tf.Tensor 'input_11:0' shape=(?, 15) dtype=float32>,
 <tf.Tensor 'input_10:0' shape=(?, 1) dtype=float32>]

In [ ]:
30000

In [88]:
anneal_function = 'logistic'
k = 0.01
x0 = int( 30000000 / (6400 * 2))

In [89]:
def kl_anneal_function(anneal_function, step, k, x0):
    if anneal_function == 'logistic':
        return float(1/(1+np.exp(-k*(step-x0))))
    elif anneal_function == 'linear':
        return min(1, step/x0/2)

In [90]:
import matplotlib.pyplot as plt
x = [kl_anneal_function(anneal_function, i, k, x0) for i in range(int(30000000 / 6400))]
plt.plot(x)
plt.ylabel('some numbers')
plt.show()

In [91]:
x

[6.675446028932423e-11,
 6.742535376880003e-11,
 6.810298983983958e-11,
 6.878743626661415e-11,
 6.947876149433726e-11,
 7.017703465610752e-11,
 7.08823255798227e-11,
 7.159470479516345e-11,
 7.231424354064434e-11,
 7.304101377074007e-11,
 7.377508816307873e-11,
 7.451654012571177e-11,
 7.526544380445303e-11,
 7.602187409029419e-11,
 7.678590662689468e-11,
 7.755761781814429e-11,
 7.833708483580575e-11,
 7.912438562722981e-11,
 7.991959892315218e-11,
 8.072280424556455e-11,
 8.153408191566905e-11,
 8.235351306190842e-11,
 8.318117962807981e-11,
 8.401716438153009e-11,
 8.486155092143066e-11,
 8.57144236871397e-11,
 8.657586796664388e-11,
 8.744596990508957e-11,
 8.832481651339506e-11,
 8.92124956769541e-11,
 9.010909616442247e-11,
 9.101470763659567e-11,
 9.192942065537623e-11,
 9.285332669282755e-11,
 9.378651814032393e-11,
 9.472908831778707e-11,
 9.56811314830208e-11,
 9.664274284113463e-11,
 9.761401855406532e-11,
 9.859505575019419e-11,
 9.958595253405761e-11,
 1.0058680799616041e

In [47]:
x

[0.04742587317756678,
 0.04765226830745615,
 0.04787968984991887,
 0.048108141938872764,
 0.04833762871990521,
 0.048568154350256104,
 0.048799722998799984,
 0.04903233884602757,
 0.049266006084026655,
 0.049500728916462365,
 0.04973651155855672,
 0.04997335823706753,
 0.05021127319026653,
 0.05045026066791703,
 0.050690324931250655,
 0.05093147025294352,
 0.051173700917091655,
 0.05141702121918567,
 0.051661435466084836,
 0.05190694797599032,
 0.05215356307841774,
 0.05240128511416888,
 0.0526501184353029,
 0.05290006740510655,
 0.05315113639806372,
 0.05340332979982423,
 0.05365665200717188,
 0.053911107427991614,
 0.05416670048123607,
 0.0544234355968911,
 0.05468131721594076,
 0.05494034979033125,
 0.0552005377829343,
 0.05546188566750936,
 0.05572439792866542,
 0.05598807906182166,
 0.05625293357316737,
 0.05651896597962102,
 0.056786180808788486,
 0.05705458259892042,
 0.05732417589886873,
 0.05759496526804217,
 0.05786695527636101,
 0.05814015050421101,
 0.05841455554239623,
 0.

In [42]:
for i in range(100):
    kl_inputs = np.array([kl_anneal_function(anneal_function, i, k, x0)] * len(q_july))
    hist = run.model.fit([q_july, q_july, kl_inputs], [np.expand_dims(q_july, axis=-1), np.ones((len(q_july), 16))], verbose=0, epochs=1)
    
    print(i, eval_july(run.encoder), hist.history['rec_loss'], hist.history['kl_loss'])

ResourceExhaustedError: OOM when allocating tensor with shape[32,15,50512]
	 [[Node: training_3/Adam/gradients/rec_3/Max_grad/Equal = Equal[T=DT_FLOAT, _class=["loc:@rec_3/Max"], _device="/job:localhost/replica:0/task:0/gpu:0"](training_3/Adam/gradients/rec_3/Max_grad/Reshape, rec_3/add)]]
	 [[Node: training_3/Adam/gradients/gru_8/while/mul_3_grad/Reshape_1/_2671 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_584_training_3/Adam/gradients/gru_8/while/mul_3_grad/Reshape_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](^_clooptraining_3/Adam/gradients/AddN_7/_2344)]]

Caused by op 'training_3/Adam/gradients/rec_3/Max_grad/Equal', defined at:
  File "/usr/local/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code


In [10]:
def generate_output(model, bpe, x, nrows=100000, idx=None):

    gen_x = np.argmax(model.predict(x), axis=-1) if idx == None else np.argmax(model.predict(x)[idx], axis=-1)
    bleu = []
    results = []
    count = 0
    for i, k in zip(gen_x, x[0]):
        gen_x = " ".join([bpe.index2word[t] for t in i])
        real_x = " ".join([bpe.index2word[t] for t in k])

        bleu.append(sentence_bleu(real_x, gen_x))
        
        real_x = real_x.replace("▁the", "")
        real_x = real_x.replace("▁","")
        gen_x = gen_x.replace("▁the", "")
        gen_x = gen_x.replace("▁","")
        
        if count > nrows:
            break
        count = count + 1
        results.append("%s\t%s" % (real_x, gen_x))
        
    
    print("BLEU: %.4f" % np.mean(bleu))
    idx = np.arange(len(results))
    shuffle(idx)
    for i in idx[:nrows]:
        print(results[i])

In [19]:
generate_output(run.model, bpe, [q_july, q_july, np.ones((len(q_july)))], idx=0)

BLEU: 0.6193
great smoky mountains national park          	great smoky mountains national park park park park park park park park park park park
radio city music hall           	radio city music hall hall hall hall hall hall hall hall hall hall hall hall
che vy monte carlo           	che vy monte carlo carlo carlo carlo carlo carlo carlo carlo carlo carlo carlo carlo
types of cer ner certifications          	types of cer ner certifications certifications certifications certifications certifications certifications certifications certifications certifications certifications certifications
dawn of  planet of  apes review       	dawn of  planet of  apes review review review review review review review review
removing smoke smell from house          	removing smoke smell from house house house house house house house house house house house
cord of wood dimensions           	cord of wood dimensions dimensions dimensions dimensions dimensions dimensions dimensions dimensions dimensions dimen

In [36]:
from sklearn.metrics.pairwise import euclidean_distances
from scipy.stats import wasserstein_distance
def eval_july(run, idx=None):
    q_ = run.predict(q_july)[idx] if idx != None else run.predict(q_july)
    d_ = run.predict(d_july)[idx] if idx != None else run.predict(d_july)
    cosine = CosineSim(q_.shape[-1])
    
    pred = []
    for i in range(len(q_)):
        pred.append(wasserstein_distance(q_[i], d_[i]))
    pred = np.array(pred)
    cos_pred = cosine.model.predict([q_, d_])
    return auc(qrel_july, pred.flatten()), auc(qrel_july, cos_pred.flatten())

(426, 16)


0.520847695078832